In [1]:
from utilities.dataloader import VOC
from imgaug import augmenters as iaa

import torchvision.transforms as transform
from utilities.augmentation import Augmenter

from torch.utils.data import dataloader
import torch
import torch.nn as nn

import yolov1_simple
from yolov1_simple import detection_loss_4_yolo

from utilities.dataloader import detection_collate
from utilities.utils import save_checkpoint

#print(torch.cuda.is_available())

import time

import os



#data_path = "/home/ali/VOCdevkit/VOC2012"
#class_path = "/home/ali/VOCdevkit/VOC2012/voc.names"
data_path = "/home/ali/VOCdevkit/VOC2012_1500"
class_path = "/home/ali/VOCdevkit/VOC2012_1500/voc.names"

batch_size = 10
#device = 'cpu'
device = 'cuda'
check_point_path = "/home/ali/VOCdevkit/VOC2012_1500/save_model_simple/"

use_visdom=True

seq = iaa.Sequential([])
compsed = transform.Compose([Augmenter(seq)])

with open(class_path) as f:
    class_list = f.read().splitlines()

train_data = VOC(root=data_path, transform=compsed, class_path=class_path)

train_loader = dataloader.DataLoader(dataset=train_data,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     collate_fn=detection_collate)

# model


dropout = 0.4
num_class = 20
learning_rate = .4
num_epochs = 100

net = yolov1_simple.YOLOv1(params={"dropout": dropout,
                            "num_class": num_class})


if device == 'cpu':
    model = nn.DataParallel(net).cpu()
else:
    model=torch.nn.DataParallel(net).cuda()



optimaizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                              weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimaizer, gamma=0.95)

total_step = len(train_loader)

# print(total_step)

total_train_step = num_epochs * total_step

for epoch in range(1, num_epochs + 1):

    if (epoch == 200) or (epoch == 400):
        scheduler.step()
        
    start = time.time()
    for i, (images, labels, sizes) in enumerate(train_loader):
        current_train_step = (epoch - 1) * total_step + i + 1
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        # calc Loss
        loss, \
        obj_coord1_loss, \
        obj_size1_loss, \
        obj_class_loss, \
        noobjness1_loss, \
        objness1_loss = detection_loss_4_yolo(outputs, labels, device)
        # print(current_train_step)

        # backward and optimze
        optimaizer.zero_grad()
        loss.backward()
        optimaizer.step()
        
        end = time.time()
        print(
            'epoch: [{}/{}], total step:[{}/{}] , batchstep [{}/{}], lr: {},'
            'total_loss: {:.4f}, objness1: {:.4f}, class_loss: {:.4f} Time: {:0.3f} seconds'.format(epoch, num_epochs,
                                                                              current_train_step, total_train_step,
                                                                              i + 1, total_step, learning_rate,
                                                                              loss.item(), obj_coord1_loss,
                                                                              obj_size1_loss, obj_class_loss, end-start))
        start = time.time()
     
                                                                        


    if (epoch %  20 == 0):
        '''
        torch.save({'test': epoch}, 'cc.zip')
        print("Saved...")

        '''
        save_checkpoint({'epoch':epoch+1,
                             'arch': "YOLOv1",
                             'state_dict':model.state_dict(),
                             },False, filename=os.path.join(check_point_path,'ep{:05d}_loss{:.04f}_lr{}.pth.tar'.format(epoch,
                                                            loss.item(),learning_rate,))
                            )
        print("The check point is saved")




Image Folder : /home/ali/VOCdevkit/VOC2012_1500/JPEGImages
Label Folder : /home/ali/VOCdevkit/VOC2012_1500/Annotations



VOC Parsing:   |████████████████████████████████████████| 100.0% (1500/1500)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (1500/1500)  Complete



/home/ali/Ali_Files/Yolo_July/YOLO_V1_B/utilities/augmentation.py:25: DeprecationWarning: Method `BoundingBoxesOnImage.cut_out_of_image()` is deprecated. Use `BoundingBoxesOnImage.clip_out_of_image()` instead. clip_out_of_image() has the exactly same interface.
  bbs_aug = bbs_aug.remove_out_of_image().cut_out_of_image()


epoch: [1/100], total step:[1/15000] , batchstep [1/150], lr: 0.4,total_loss: 30.2235, objness1: 6.7826, class_loss: 8.9846 Time: 8.108 seconds
epoch: [1/100], total step:[2/15000] , batchstep [2/150], lr: 0.4,total_loss: 8135631175680.0000, objness1: 3604452737024.0000, class_loss: 4531178962944.0000 Time: 19.700 seconds
epoch: [1/100], total step:[3/15000] , batchstep [3/150], lr: 0.4,total_loss: 16305801396224.0000, objness1: 8543887949824.0000, class_loss: 7761913446400.0000 Time: 29.900 seconds
epoch: [1/100], total step:[4/15000] , batchstep [4/150], lr: 0.4,total_loss: 600320442368.0000, objness1: 319042945024.0000, class_loss: 281277530112.0000 Time: 21.600 seconds
epoch: [1/100], total step:[5/15000] , batchstep [5/150], lr: 0.4,total_loss: 1083758673920.0000, objness1: 519955709952.0000, class_loss: 563802996736.0000 Time: 19.700 seconds
epoch: [1/100], total step:[6/15000] , batchstep [6/150], lr: 0.4,total_loss: 422990577664.0000, objness1: 215958536192.0000, class_loss: 20

epoch: [1/100], total step:[48/15000] , batchstep [48/150], lr: 0.4,total_loss: 11636490240.0000, objness1: 5872076800.0000, class_loss: 5764413952.0000 Time: 32.748 seconds
epoch: [1/100], total step:[49/15000] , batchstep [49/150], lr: 0.4,total_loss: 12746358784.0000, objness1: 6602432000.0000, class_loss: 6143927296.0000 Time: 36.487 seconds
epoch: [1/100], total step:[50/15000] , batchstep [50/150], lr: 0.4,total_loss: 23135092736.0000, objness1: 11355579392.0000, class_loss: 11779513344.0000 Time: 22.808 seconds
epoch: [1/100], total step:[51/15000] , batchstep [51/150], lr: 0.4,total_loss: 10716462080.0000, objness1: 5069145600.0000, class_loss: 5647315968.0000 Time: 35.440 seconds
epoch: [1/100], total step:[52/15000] , batchstep [52/150], lr: 0.4,total_loss: 8983268352.0000, objness1: 4501762560.0000, class_loss: 4481505280.0000 Time: 22.402 seconds
epoch: [1/100], total step:[53/15000] , batchstep [53/150], lr: 0.4,total_loss: 11816943616.0000, objness1: 5266520064.0000, clas

...

epoch: [99/100], total step:[14798/15000] , batchstep [98/150], lr: 0.4,total_loss: 2683141095424.0000, objness1: 1331938459648.0000, class_loss: 1351202635776.0000 Time: 4.900 seconds
epoch: [99/100], total step:[14799/15000] , batchstep [99/150], lr: 0.4,total_loss: 2355066830848.0000, objness1: 1296511139840.0000, class_loss: 1058555822080.0000 Time: 3.900 seconds
epoch: [99/100], total step:[14800/15000] , batchstep [100/150], lr: 0.4,total_loss: 2256796123136.0000, objness1: 1085180215296.0000, class_loss: 1171615776768.0000 Time: 4.700 seconds
epoch: [99/100], total step:[14801/15000] , batchstep [101/150], lr: 0.4,total_loss: 661093941248.0000, objness1: 351306022912.0000, class_loss: 309787951104.0000 Time: 4.400 seconds
epoch: [99/100], total step:[14802/15000] , batchstep [102/150], lr: 0.4,total_loss: 1618986008576.0000, objness1: 816944971776.0000, class_loss: 802041036800.0000 Time: 4.000 seconds
epoch: [99/100], total step:[14803/15000] , batchstep [103/150], lr: 0.4,tota

epoch: [99/100], total step:[14843/15000] , batchstep [143/150], lr: 0.4,total_loss: 1389905051648.0000, objness1: 706960883712.0000, class_loss: 682944167936.0000 Time: 4.857 seconds
epoch: [99/100], total step:[14844/15000] , batchstep [144/150], lr: 0.4,total_loss: 3560959901696.0000, objness1: 2115924525056.0000, class_loss: 1445035245568.0000 Time: 7.500 seconds
epoch: [99/100], total step:[14845/15000] , batchstep [145/150], lr: 0.4,total_loss: 3292632711168.0000, objness1: 1498639499264.0000, class_loss: 1793993211904.0000 Time: 8.592 seconds
epoch: [99/100], total step:[14846/15000] , batchstep [146/150], lr: 0.4,total_loss: 4347001831424.0000, objness1: 2309205786624.0000, class_loss: 2037796306944.0000 Time: 6.700 seconds
epoch: [99/100], total step:[14847/15000] , batchstep [147/150], lr: 0.4,total_loss: 1975500800000.0000, objness1: 954221461504.0000, class_loss: 1021279272960.0000 Time: 6.900 seconds
epoch: [99/100], total step:[14848/15000] , batchstep [148/150], lr: 0.4,

epoch: [100/100], total step:[14888/15000] , batchstep [38/150], lr: 0.4,total_loss: 2200284168192.0000, objness1: 1004498911232.0000, class_loss: 1195785191424.0000 Time: 7.900 seconds
epoch: [100/100], total step:[14889/15000] , batchstep [39/150], lr: 0.4,total_loss: 3962663600128.0000, objness1: 2051237216256.0000, class_loss: 1911426514944.0000 Time: 6.600 seconds
epoch: [100/100], total step:[14890/15000] , batchstep [40/150], lr: 0.4,total_loss: 3836508897280.0000, objness1: 2009196265472.0000, class_loss: 1827312500736.0000 Time: 4.700 seconds
epoch: [100/100], total step:[14891/15000] , batchstep [41/150], lr: 0.4,total_loss: 3006398201856.0000, objness1: 1809350393856.0000, class_loss: 1197047939072.0000 Time: 5.200 seconds
epoch: [100/100], total step:[14892/15000] , batchstep [42/150], lr: 0.4,total_loss: 4196163387392.0000, objness1: 2011280179200.0000, class_loss: 2184883208192.0000 Time: 8.400 seconds
epoch: [100/100], total step:[14893/15000] , batchstep [43/150], lr: 0

epoch: [100/100], total step:[14933/15000] , batchstep [83/150], lr: 0.4,total_loss: 1266345705472.0000, objness1: 692340195328.0000, class_loss: 574005444608.0000 Time: 4.700 seconds
epoch: [100/100], total step:[14934/15000] , batchstep [84/150], lr: 0.4,total_loss: 1317464309760.0000, objness1: 587184537600.0000, class_loss: 730279837696.0000 Time: 5.400 seconds
epoch: [100/100], total step:[14935/15000] , batchstep [85/150], lr: 0.4,total_loss: 1075593019392.0000, objness1: 421713182720.0000, class_loss: 653879869440.0000 Time: 4.600 seconds
epoch: [100/100], total step:[14936/15000] , batchstep [86/150], lr: 0.4,total_loss: 1525007646720.0000, objness1: 876845072384.0000, class_loss: 648162574336.0000 Time: 5.600 seconds
epoch: [100/100], total step:[14937/15000] , batchstep [87/150], lr: 0.4,total_loss: 12197090557952.0000, objness1: 5283658072064.0000, class_loss: 6913433010176.0000 Time: 8.300 seconds
epoch: [100/100], total step:[14938/15000] , batchstep [88/150], lr: 0.4,tota

epoch: [100/100], total step:[14977/15000] , batchstep [127/150], lr: 0.4,total_loss: 7953381326848.0000, objness1: 3935409799168.0000, class_loss: 4017971265536.0000 Time: 12.024 seconds
epoch: [100/100], total step:[14978/15000] , batchstep [128/150], lr: 0.4,total_loss: 5585774313472.0000, objness1: 3788479135744.0000, class_loss: 1797295308800.0000 Time: 8.000 seconds
epoch: [100/100], total step:[14979/15000] , batchstep [129/150], lr: 0.4,total_loss: 6558224744448.0000, objness1: 2828090998784.0000, class_loss: 3730133745664.0000 Time: 6.200 seconds
epoch: [100/100], total step:[14980/15000] , batchstep [130/150], lr: 0.4,total_loss: 4870002704384.0000, objness1: 2272894124032.0000, class_loss: 2597108580352.0000 Time: 5.700 seconds
epoch: [100/100], total step:[14981/15000] , batchstep [131/150], lr: 0.4,total_loss: 2531717021696.0000, objness1: 1235669876736.0000, class_loss: 1296047144960.0000 Time: 6.300 seconds
epoch: [100/100], total step:[14982/15000] , batchstep [132/150]